In [1]:
import numpy as np
import pandas as pd

#inputs
notional = 100000000
coupon = 0.055
term = 36 
CDR = 0.02
CPR = 0.25
severity = 0.25
advance_rate = 0.95
facility_rate = 0.02


In [2]:
#initialisation


def sch_pay(p, t, r):
    return p*r*((1+r)**t)/((1+r)**t-1)


def calculate_assets():
    begin_col_bal = []
    surv_sch_payment = []
    surv_sch_principal = []
    defaults = []
    recovery = []
    net_loss = []
    rem_col_bal = []
    sch_payment = []
    sch_interest = []
    sch_principal = [] 
    prepayment = []
    principal_paid = []
    cashflow = []
    end_col_bal = []
    begin_fac_bal = []


    monthly_default_rate = 1 - (1 - CDR)**(1/12)
    monthly_prepayment_rate = 1 - (1 - CPR)**(1/12)

    begin_col_bal.append(notional)
    begin_fac_bal.append(advance_rate * notional)


    for i in range(term):
        surv_sch_payment.append( sch_pay( begin_col_bal[i], term - i, coupon/12 ) )
        surv_sch_principal.append( surv_sch_payment[i] - begin_col_bal[i] * coupon/12 )
        defaults.append( monthly_default_rate * begin_col_bal[i] )
        recovery.append( defaults[i] * (1 - severity ) )
        net_loss.append( defaults[i] - recovery[i] )
        rem_col_bal.append( begin_col_bal[i] - defaults[i] )
        sch_payment.append( sch_pay( rem_col_bal[i], term - i, coupon/12))
        sch_interest.append( rem_col_bal[i] * coupon/12)
        sch_principal.append( sch_payment[i] - sch_interest[i] )
        prepayment.append( (begin_col_bal[i] - surv_sch_principal[i]) * monthly_prepayment_rate )
        principal_paid.append( recovery[i] + prepayment[i] + sch_principal[i] )
        cashflow.append( principal_paid[i] + sch_interest[i] )
        end_col_bal.append( begin_col_bal[i] - principal_paid[i] - net_loss[i])
        begin_col_bal.append( end_col_bal[i] )
        
    data = {
    "Beginning collateral balance": begin_col_bal[0:term],
    "Surviving Scheduled payment": surv_sch_payment,
    "Surviving scheduled principal": surv_sch_principal,
    "Defaults": defaults,
    "Recovery": recovery,
    "Net loss": net_loss,
    "Remaining Collateral balance": rem_col_bal,
    "Scheduled payment": sch_payment,
    "scheduled interest": sch_interest,
    "scheduled principal": sch_principal,
    "prepayment": prepayment,
    "principal paid": principal_paid,
    "cashflow": cashflow,
    "ending collateral balance": end_col_bal
    }

    assets = pd.DataFrame(data)
    pd.set_option('display.float_format', '{:,.2f}'.format)
    
    return assets

    

In [3]:
assets = calculate_assets()

In [4]:
assets

,Beginning collateral balance,Surviving Scheduled payment,Surviving scheduled principal,Defaults,Recovery,Net loss,Remaining Collateral balance,Scheduled payment,scheduled interest,scheduled principal,prepayment,principal paid,cashflow,ending collateral balance
0,"100,000,000.00","3,019,590.18","2,561,256.85","168,214.26","126,160.69","42,053.56","99,831,785.74","3,014,510.80","457,562.35","2,556,948.45","2,308,170.28","4,991,279.42","5,448,841.77","94,966,667.01"
1,"94,966,667.01","2,942,981.47","2,507,717.58","159,747.47","119,810.60","39,936.87","94,806,919.54","2,938,030.95","434,531.71","2,503,499.24","2,190,206.82","4,813,516.66","5,248,048.37","90,113,213.49"
2,"90,113,213.49","2,868,316.36","2,455,297.46","151,583.27","113,687.45","37,895.82","89,961,630.22","2,863,491.44","412,324.14","2,451,167.30","2,076,477.90","4,641,332.66","5,053,656.80","85,433,985.01"
3,"85,433,985.01","2,795,545.55","2,403,973.12","143,712.14","107,784.11","35,928.04","85,290,272.87","2,790,843.04","390,913.75","2,399,929.29","1,966,850.15","4,474,563.54","4,865,477.29","80,923,493.44"
4,"80,923,493.44","2,724,620.97","2,353,721.63","136,124.85","102,093.64","34,031.21","80,787,368.58","2,720,037.77","370,275.44","2,349,762.33","1,861,194.09","4,313,050.06","4,683,325.50","76,576,412.17"
5,"76,576,412.17","2,655,495.79","2,304,520.57","128,812.44","96,609.33","32,203.11","76,447,599.72","2,651,028.87","350,384.83","2,300,644.04","1,759,384.08","4,156,637.45","4,507,022.28","72,387,571.61"
6,"72,387,571.61","2,588,124.36","2,256,347.99","121,766.21","91,324.66","30,441.55","72,265,805.40","2,583,770.77","331,218.27","2,252,552.49","1,661,298.18","4,005,175.33","4,336,393.60","68,351,954.73"
7,"68,351,954.73","2,522,462.18","2,209,182.39","114,977.73","86,233.30","28,744.43","68,236,977.00","2,518,219.04","312,752.81","2,205,466.23","1,566,818.05","3,858,517.58","4,171,270.39","64,464,692.72"
8,"64,464,692.72","2,458,465.88","2,163,002.71","108,438.80","81,329.10","27,109.70","64,356,253.92","2,454,330.39","294,966.16","2,159,364.23","1,475,828.86","3,716,522.19","4,011,488.36","60,721,060.83"
9,"60,721,060.83","2,396,093.21","2,117,788.35","102,141.48","76,606.11","25,535.37","60,618,919.35","2,392,062.64","277,836.71","2,114,225.93","1,388,219.18","3,579,051.22","3,856,887.93","57,116,474.24"


In [5]:
assets.to_excel("Amortizing loan calculator.xlsx")